# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2021DL folder and put all the files under A0 folder, then '2021DL/A0'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DL2021/midterm2022/DataPrepare'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "UTC"
time.tzset()

from pytorch_autograd_and_nn_sol_new import * # select your function
from a4_helper import *
hello()

py_path = os.path.join(GOOGLE_DRIVE_PATH, 'pytorch_autograd_and_nn_sol_new.py') # select your function
py_edit_time = time.ctime(os.path.getmtime(py_path))
print('pytorch_autograd_and_nn.py last edited on %s' % py_edit_time)

!pip install pytorch-lightning --quiet
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR100
import matplotlib.pyplot as plt

# CIFAR-100 Dataset

In [ ]:
class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir: str = './data'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        
        self.transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.RandomRotation(15),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
                                                  ])
        
        self.transform_test = transforms.Compose([transforms.ToTensor(),
                                                    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
                                                  ])

        
    def prepare_data(self):
        # download 
        torchvision.datasets.CIFAR100(self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR100(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            cifar_full = torchvision.datasets.CIFAR100(self.data_dir, train=True, transform=self.transform_train)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.cifar_test = torchvision.datasets.CIFAR100(self.data_dir, train=False, transform=self.transform_test)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=16, num_workers=2)

In [ ]:
# Init our data pipeline
dm = CIFAR100DataModule(batch_size=256)

# To access the x_dataloader we need to call prepare_data and setup.
dm.prepare_data()

dm.setup()

testloader = dm.test_dataloader()
valloader = dm.val_dataloader()
trainloader = dm.train_dataloader()

In [ ]:
#function to read files present in the Python version of the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

metaData = unpickle('/content/data/cifar-100-python/meta')
label_names = metaData['fine_label_names']
print('Number of class is ', len(label_names))

# get some random test images to see if things are getting displayed correctly
dataiter = iter(trainloader)
images, labels = dataiter.next()
labels_list = labels.tolist()

plt_need_idx = 5
print( 'Index of corresponding label is ', labels_list[plt_need_idx] )
print( 'Figure size is ', images[plt_need_idx].numpy().shape )
print( 'Corresponding label is ', label_names[labels_list[plt_need_idx]] )
plt.imshow(np.transpose(images[plt_need_idx].numpy(), (1, 2, 0)))

# New Train Model (change train_part345 to train_part_NEW)

In [ ]:
def check_accuracy_part_NEW(loader, model):  
  num_correct = 0
  num_samples = 0
  model.eval()  # set model to evaluation mode
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device='cuda', dtype=to_float)  # move to device, e.g. GPU
      y = y.to(device='cuda', dtype=to_long)
      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
  return acc


def adjust_learning_rate(optimizer, lrd, epoch, schedule):
  """
  Multiply lrd to the learning rate if epoch is in schedule
  
  Inputs:
  - optimizer: An Optimizer object we will use to train the model
  - lrd: learning rate decay; a factor multiplied at scheduled epochs
  - epochs: the current epoch number
  - schedule: the list of epochs that requires learning rate update
  
  Returns: Nothing, but learning rate might be updated
  """
  if epoch in schedule:
    for param_group in optimizer.param_groups:
      print('lr decay from {} to {}'.format(param_group['lr'], param_group['lr'] * lrd))
      param_group['lr'] *= lrd


def train_part_NEW(model, optimizer, epochs=1, learning_rate_decay=.1, schedule=[], verbose=True):
  """
  Train a model on CIFAR-10 using the PyTorch Module API.
  
  Inputs:
  - model: A PyTorch Module giving the model to train.
  - optimizer: An Optimizer object we will use to train the model
  - epochs: (Optional) A Python integer giving the number of epochs to train for
  
  Returns: Nothing, but prints model accuracies during training.
  """
  model = model.to(device='cuda')  # move the model parameters to CPU/GPU
  num_iters = epochs * len(trainloader)
  print_every = 100
  if verbose:
    num_prints = num_iters // print_every + 1
  else:
    num_prints = epochs
  acc_history = torch.zeros(num_prints, dtype=to_float)
  iter_history = torch.zeros(num_prints, dtype=to_long)
  for e in range(epochs):
    
    adjust_learning_rate(optimizer, learning_rate_decay, e, schedule)
    
    for t, (x, y) in enumerate(trainloader):
      model.train()  # put model to training mode
      x = x.to(device='cuda', dtype=to_float)  # move to device, e.g. GPU
      y = y.to(device='cuda', dtype=to_long)

      scores = model(x)
      loss = F.cross_entropy(scores, y)

      # Zero out all of the gradients for the variables which the optimizer
      # will update.
      optimizer.zero_grad()

      # This is the backwards pass: compute the gradient of the loss with
      # respect to each  parameter of the model.
      loss.backward()

      # Actually update the parameters of the model using the gradients
      # computed by the backwards pass.
      optimizer.step()

      tt = t + e * len(trainloader)
      ee = e + 1
      if verbose and (tt % print_every == 0 or (e == epochs-1 and t == len(trainloader)-1)):
        print('Epoch %d, Iteration %d, loss = %.4f' % (ee, tt, loss.item()))
        acc = check_accuracy_part_NEW(valloader, model)
        acc_history[tt // print_every] = acc
        iter_history[tt // print_every] = tt
        print()
      elif not verbose and (t == len(trainloader)-1):
        print('Epoch %d, Iteration %d, loss = %.4f' % (ee, tt, loss.item()))
        acc = check_accuracy_part_NEW(valloader, model)
        acc_history[e] = acc
        iter_history[e] = tt
        print()

  torch.save(model.state_dict(), "/content/drive/MyDrive/DL2021/midterm2022/DataPrepare/saveNEW.pt")
  print('Save model weight')

  return acc_history, iter_history

# Start Training

In [ ]:
# Init our model
names = ['resnet47']

name = names[-1]
reset_seed(0)
print(name, '\n')
model = get_resnet(name)
  
# optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=.9, weight_decay=1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

acc_history, iter_history = train_part_NEW(model, optimizer, epochs=20, schedule=[10, 20, 40, 80], verbose=False)
acc_history_dict[name] = acc_history
iter_history_dict[name] = iter_history

# Validation Curve

In [ ]:
plt.title('Val accuracies')
for name in names:
  plt.plot(iter_history_dict[name], acc_history_dict[name], '-o')
plt.legend(names, loc='upper left')
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.gcf().set_size_inches(9, 4)
plt.show()

# Confusion Matrix

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import *

classes = ['beaver', 'dolphin', 'otter', 'seal', 'whale', 
'aquarium' ,'fish', 'ray', 'shark', 'trout', 
'orchids', 'poppies', 'roses', 'sunflowers', 'tulips', 
'bottles', 'bowls', 'cans', 'cups', 'plates', 
'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers', 
'clock', 'computer keyboard', 'lamp', 'telephone', 'television', 'bed', 'chair', 'couch', 'table', 'wardrobe', 
'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach', 
'bear', 'leopard', 'lion', 'tiger', 'wolf', 
'bridge', 'castle', 'house', 'road', 'skyscraper', 
'cloud', 'forest', 'mountain', 'plain', 'sea', 
'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo', 
'fox', 'porcupine', 'possum', 'raccoon', 'skunk', 
'crab', 'lobster', 'snail', 'spider', 'worm', 
'baby', 'boy', 'girl', 'man', 'woman', 
'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle', 
'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel', 
'maple', 'oak', 'palm', 'pine', 'willow', 
'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train', 
'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor']

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device: ",device)
print(torch.cuda.get_device_name())

def test_label_predictions(model, device, testloader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            prediction = output.argmax(dim=1, keepdim=True)
            actuals.extend(target.view_as(prediction))
            predictions.extend(prediction)
    return [i.item() for i in actuals], [i.item() for i in predictions]

actualsTrain, predictionsTrain = test_label_predictions(model, device, trainloader)
print('Train Dataset Accuracy: %f' % accuracy_score(actualsTrain, predictionsTrain))

actualsVal, predictionsVal = test_label_predictions(model, device, valloader)
print('Val. Dataset Accuracy: %f' % accuracy_score(actualsVal, predictionsVal))

actualsTest, predictionsTest = test_label_predictions(model, device, testloader)
print('Test Dataset Accuracy: %f' % accuracy_score(actualsTest, predictionsTest))

In [ ]:
cm = confusion_matrix(actualsTest, predictionsTest)
print(cm)
fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(211)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + classes)
ax.set_yticklabels([''] + classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()